In [1]:
from sklearn.datasets import make_classification
import numpy as np
import time
import faiss

import torch

In [2]:
# X, y = make_classification(
#     n_samples=100000, n_features=4, n_classes=4, n_clusters_per_class=2, random_state=1024, n_informative=4, n_redundant = 0
# )
features = 256
clusters = 256
X, y = make_classification(
    n_samples=100, n_features=features, n_classes=2, random_state=1024, n_informative=features, n_redundant=0, n_clusters_per_class = 1
)
X.shape, y.shape

((100, 256), (100,))

In [3]:
xy = np.concatenate((X, y.reshape(-1, 1)), axis=1)
# np.savetxt("test_clusters_" + str(clusters) + ".csv", xy, delimiter=",")
np.savetxt("test_samples.csv", xy, delimiter=",")

In [137]:
from fast_pytorch_kmeans import KMeans

kmeans = KMeans(n_clusters=clusters, mode='euclidean', verbose=1, tol = -1, max_iter=50)
x = torch.from_numpy(X).to("cuda")

start_time = time.time()
labels = kmeans.fit_predict(x)
# 记录结束时间
end_time = time.time()

# 计算并输出fit方法所需的时间
fit_time = end_time - start_time
print(f"KMeans fit time: {fit_time * 1000} ms")

used 50 iterations (17.7545s) to cluster 100000 items into 256 clusters
KMeans fit time: 17754.658460617065 ms


In [138]:
d = X.shape[1]
clus = faiss.Clustering(d, clusters)
clus.verbose = True
clus.niter = 50
# otherwise the kmeans implementation sub-samples the training set
clus.max_points_per_centroid = 10**9
cfg = faiss.GpuIndexFlatConfig()
cfg.useFloat16 = False
cfg.device = 0
index = faiss.GpuIndexFlatL2(faiss.StandardGpuResources(), d, cfg)
start_time = time.time()
clus.train(X, index)
# 记录结束时间
end_time = time.time()
#计算并输出fit方法所需的时间
fit_time = end_time - start_time
print(f"KMeans fit time: {fit_time * 1000} ms")

Clustering 100000 points in 1024D to 256 clusters, redo 1 times, 50 iterations
  Preprocessing in 0.04 s
KMeans fit time: 3554.8036098480225 ms: objective=3.40156e+10 imbalance=1.259 nsplit=0       



In [134]:
# ncentroids = clusters
# niter = 50
# verbose = True
# d = X.shape[1]
# kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose, nredo = 1, gpu=True)
# kmeans.max_points_per_centroid = 10**9
# start_time = time.time()
# kmeans.train(X)
# # 记录结束时间
# end_time = time.time()
# # 计算并输出fit方法所需的时间
# fit_time = end_time - start_time
# print(f"KMeans fit time: {fit_time * 1000} ms")